## Imports and Setup

This notebook will load the raw files from Data Mart and clean it.
Student Success Services (term-level) → aggregate to annual
Program Awards (4 files) → aggregate to total institutional awards
Student Headcount (annual) → for normalization
FTES (annual) → for alternative normalization

master_dataset_annual.csv - Merged dataset ready for analysis with normalized rates

In [1]:
#Import required libraries. pandas for data manipulation and numpy for numerical operations and handling missing data

import pandas as pd
import numpy as np

# Configure pandas display options for better visibility
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

print("Libraries imported successfully!")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

Libraries imported successfully!
pandas version: 2.3.0
numpy version: 1.26.4


## Load and inspect the data

In [2]:
# Load the Student Success Services file
services_file = "fall2014_fall2025_StudentSuccessServiceSumm.csv"
df_services_raw = pd.read_csv(services_file)

# Check the shape (rows, columns)
print("Shape:", df_services_raw.shape)
print("This means:", df_services_raw.shape[0], "rows ×", df_services_raw.shape[1], "columns")
print("\n" + "=" * 60)

# Display first 10 rows to see the structure
print("\nFirst 10 rows:")
df_services_raw.head(10)



Shape: (14, 130)
This means: 14 rows × 130 columns


First 10 rows:


,Unnamed: 0,Unnamed: 1,Fall 2014,Fall 2014.1,Fall 2014.2,Fall 2014.3,Spring 2015,Spring 2015.1,Spring 2015.2,Spring 2015.3,Summer 2015,Summer 2015.1,Summer 2015.2,Summer 2015.3,Fall 2015,Fall 2015.1,Fall 2015.2,Fall 2015.3,Spring 2016,Spring 2016.1,Spring 2016.2,Spring 2016.3,Summer 2016,Summer 2016.1,Summer 2016.2,Summer 2016.3,Fall 2016,Fall 2016.1,Fall 2016.2,Fall 2016.3,Spring 2017,Spring 2017.1,Spring 2017.2,Spring 2017.3,Summer 2017,Summer 2017.1,Summer 2017.2,Summer 2017.3,Fall 2017,Fall 2017.1,Fall 2017.2,Fall 2017.3,Spring 2018,Spring 2018.1,Spring 2018.2,Spring 2018.3,Summer 2018,Summer 2018.1,Summer 2018.2,Summer 2018.3,Fall 2018,Fall 2018.1,Fall 2018.2,Fall 2018.3,Spring 2019,Spring 2019.1,Spring 2019.2,Spring 2019.3,Summer 2019,Summer 2019.1,Summer 2019.2,Summer 2019.3,Fall 2019,Fall 2019.1,Fall 2019.2,Fall 2019.3,Spring 2020,Spring 2020.1,Spring 2020.2,Spring 2020.3,Summer 2020,Summer 2020.1,Summer 2020.2,Summer 2020.3,Fall 2020,Fall 2020.1,Fall 2020.2,Fall 2020.3,Spring 2021,Spring 2021.1,Spring 2021.2,Spring 2021.3,Summer 2021,Summer 2021.1,Summer 2021.2,Summer 2021.3,Fall 2021,Fall 2021.1,Fall 2021.2,Fall 2021.3,Spring 2022,Spring 2022.1,Spring 2022.2,Spring 2022.3,Summer 2022,Summer 2022.1,Summer 2022.2,Summer 2022.3,Fall 2022,Fall 2022.1,Fall 2022.2,Fall 2022.3,Spring 2023,Spring 2023.1,Spring 2023.2,Spring 2023.3,Summer 2023,Summer 2023.1,Summer 2023.2,Summer 2023.3,Fall 2023,Fall 2023.1,Fall 2023.2,Fall 2023.3,Spring 2024,Spring 2024.1,Spring 2024.2,Spring 2024.3,Fall 2024,Fall 2024.1,Fall 2024.2,Fall 2024.3,Spring 2025,Spring 2025.1,Spring 2025.2,Spring 2025.3,Summer 2025,Summer 2025.1,Summer 2025.2,Summer 2025.3
0,NaN,NaN,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted,Directed,Directed,Exempted,Exempted
1,NaN,NaN,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service Not Received,Service Received,Service 

The raw dataset contains 14 rows and 130 columns, indicating a wide, report-style structure rather than an analysis-ready format. The first few rows contain metadata, and the actual service-level data begins around row 4. Column names follow a repeating pattern by academic term, such as Fall 2014, Fall 2014.1, Fall 2014.2, and Fall 2014.3, with multiple columns representing different service categories for each term.

This structure confirms that the dataset requires substantial preprocessing before analysis. To prepare the data, I removed metadata rows, standardized service names, selected only the “Service Received” columns, converted string-based counts into numeric values, and reshaped the dataset from wide to long format.

## Data Cleaning: Remove Metadata Rows & Identify Service Column

In [3]:
# STEP 1: Remove the first 3 rows
# These rows contain metadata, not real data
df_services = df_services_raw.iloc[3:].copy()

# STEP 2: Reset the index
# This makes the row numbers start from 0 again
df_services = df_services.reset_index(drop=True)

# STEP 3: Rename the service column
# The service names are stored in the second column ("Unnamed: 1")
df_services = df_services.rename(columns={df_services.columns[1]: "service"})

# CHECK THE RESULT

# Show the new shape of the dataset
print("Shape after removing metadata:", df_services.shape)

# Show column names
print("\nFirst column name:", df_services.columns[0])
print("Second column name (service):", df_services.columns[1])

print("\n" + "=" * 60)

# Show the service names to confirm metadata is gone
print("Service names (you should see real services now):")
print(df_services["service"].tolist())

print("\n" + "=" * 60)

# Display the first few rows
print("First few rows of cleaned data:")
df_services.head()


Shape after removing metadata: (11, 130)

First column name: Unnamed: 0
Second column name (service): service

Service names (you should see real services now):
['Credit - Academic/Progress Probation Services                              ', 'Credit - Counseling/Advisement Services                                    ', 'Credit - Education Plan Services                                           ', 'Credit - Initial Assessment Services Placement                             ', 'Credit - Initial Orientation Services                                      ', 'Credit - Other Services                                                    ', 'Non-Credit - Counseling/Advisement Services                                ', 'Non-Credit - Education Plan Services                                       ', 'Non-Credit - Initial Assessment Services Placement                         ', 'Non-Credit - Initial Orientation Services                                  ', 'Non-Credit - Other Services                    

,Unnamed: 0,service,Fall 2014,Fall 2014.1,Fall 2014.2,Fall 2014.3,Spring 2015,Spring 2015.1,Spring 2015.2,Spring 2015.3,Summer 2015,Summer 2015.1,Summer 2015.2,Summer 2015.3,Fall 2015,Fall 2015.1,Fall 2015.2,Fall 2015.3,Spring 2016,Spring 2016.1,Spring 2016.2,Spring 2016.3,Summer 2016,Summer 2016.1,Summer 2016.2,Summer 2016.3,Fall 2016,Fall 2016.1,Fall 2016.2,Fall 2016.3,Spring 2017,Spring 2017.1,Spring 2017.2,Spring 2017.3,Summer 2017,Summer 2017.1,Summer 2017.2,Summer 2017.3,Fall 2017,Fall 2017.1,Fall 2017.2,Fall 2017.3,Spring 2018,Spring 2018.1,Spring 2018.2,Spring 2018.3,Summer 2018,Summer 2018.1,Summer 2018.2,Summer 2018.3,Fall 2018,Fall 2018.1,Fall 2018.2,Fall 2018.3,Spring 2019,Spring 2019.1,Spring 2019.2,Spring 2019.3,Summer 2019,Summer 2019.1,Summer 2019.2,Summer 2019.3,Fall 2019,Fall 2019.1,Fall 2019.2,Fall 2019.3,Spring 2020,Spring 2020.1,Spring 2020.2,Spring 2020.3,Summer 2020,Summer 2020.1,Summer 2020.2,Summer 2020.3,Fall 2020,Fall 2020.1,Fall 2020.2,Fall 2020.3,Spring 2021,Spring 2021.1,Spring 2021.2,Spring 2021.3,Summer 2021,Summer 2021.1,Summer 2021.2,Summer 2021.3,Fall 2021,Fall 2021.1,Fall 2021.2,Fall 2021.3,Spring 2022,Spring 2022.1,Spring 2022.2,Spring 2022.3,Summer 2022,Summer 2022.1,Summer 2022.2,Summer 2022.3,Fall 2022,Fall 2022.1,Fall 2022.2,Fall 2022.3,Spring 2023,Spring 2023.1,Spring 2023.2,Spring 2023.3,Summer 2023,Summer 2023.1,Summer 2023.2,Summer 2023.3,Fall 2023,Fall 2023.1,Fall 2023.2,Fall 2023.3,Spring 2024,Spring 2024.1,Spring 2024.2,Spring 2024.3,Fall 2024,Fall 2024.1,Fall 2024.2,Fall 2024.3,Spring 2025,Spring 2025.1,Spring 2025.2,Spring 2025.3,Summer 2025,Summer 2025.1,Summer 2025.2,Summer 2025.3
0,NaN,Credit - Academic/Progress Probation Services ...,16,NaN,NaN,NaN,152,NaN,NaN,NaN,71,NaN,NaN,NaN,176,NaN,NaN,NaN,102,NaN,NaN,NaN,29,NaN,NaN,NaN,12,NaN,NaN,NaN,69,NaN,NaN,NaN,19,NaN,NaN,NaN,103,NaN,NaN,NaN,100,NaN,NaN,NaN,31,NaN,NaN,NaN,140,NaN,NaN,NaN,140,NaN,NaN,NaN,25,NaN,NaN,NaN,62,NaN,NaN,NaN,43,NaN,NaN,NaN,18,NaN,NaN,NaN,68,NaN,NaN,NaN,61,NaN,NaN,NaN,10,NaN,NaN,NaN,8,NaN,NaN,NaN,2,NaN,NaN,NaN,0,NaN,NaN,NaN,4,NaN,NaN,NaN,14,NaN,NaN,NaN,3,NaN,NaN,NaN,4,NaN,NaN,NaN,9,NaN,NaN,NaN,21,NaN,NaN,NaN,6,NaN,NaN,NaN,0,NaN,NaN,NaN
1,NaN,Credit - Counseling/Advisement Services ...,"5,973","53,990",0,0,"7,908","49,810",0,0,"3,453","14,638",0,0,"13,047","46,747",0,0,"11,637","45,932",0,0,"5,462","19,613",0,0,"5,348","38,815",0,0,"10,496","45,309",0,0,"4,481","17,384",0,0,"19,356","49,246",0,0,"10,718","48,503",29,291,"4,001","22,288",196,"1,609","11,402","45,878","1,030","4,785","10,238","44,704","1,181","8,916","3,707","18,325",532,"2,906","6,926","34,457","1,275","6,515","6,024","29,193",930,"7,347","2,311","9,529",547,"9,040","5,811","21,278","1,469","14,338","6,067","15,997","1,800","7,129","1,695","9,020",515,"3,295","5,454","15,799","1,636","7,514","3,322","16,013","1,227","7,505",245,"8,669",72,"3,146","4,984","15,556","1,690","6,993","2,707","17,353","1,064","7,528",318,"34,454",121,"41,651","5,028","43,157","1,980","46,499","3,343","23,037","1,478","11,721","7,792","30,232","3,402","14,588","4,227","39,738","2,030","12,933",328,"16,625",129,"6,440"
2,NaN,Credit - Education Plan Services ...,"3,860","56,103",0,0,"6,025","51,693",0,0,"2,787","15,304",0,0,"9,097","50,697",0,0,"8,405","49,164",0,0,"4,623","20,452",0,0,"2,013","42,150",0,0,"7,487","48,318",0,0,"2,492","19,373",0,0,"11,517","57,085",0,0,"7,844","51,377",26,294,"2,719","23,570",107,"1,698","7,937","49,343",634,"5,181","7,203","47,739",839,"9,258","2,783","19,249",348,"3,090","4,855","36,528",699,"7,091","2,292","32,925",347,"7,930",610,"11,230",149,"9,438","4,274","22,815",933,"14,874","4,379","17,685","1,158","7,771","1,467","9,248",395,"3,415",413,"20,840",110,"9,040",507,"18,828",120,"8,612",7,"8,907",3,"3,215",432,"20,108",104,"8,579",352,"19,708",92,"8,500",25,"34,747",5,"41,767",454,"47,731",176,"48,303",284,"26,096",105,"13,094","3,622","34,402","1,671","16,319",891,"43,074",433,"14,530",18,"16,935",9,"6,560"
3,NaN,Credit - Initi

In [6]:
# Build list of columns to keep
cols_to_keep = ['service']  # Start with service column

# Get all term columns (everything after 'service')
all_cols = list(df_services.columns)
term_cols = all_cols[2:]  # Skip Unnamed:0 and service

# Extract positions 0 and 2 from each group of 4
for idx, col in enumerate(term_cols):
    position_in_group = idx % 4
    if position_in_group in [0, 2]:  # Positions 0 and 2 = Service Received
        cols_to_keep.append(col)

df_services_clean = df_services[cols_to_keep].copy()

print("Original shape:", df_services.shape)
print("New shape:", df_services_clean.shape)
print("\nFirst few columns:")
print(df_services_clean.columns[:10].tolist())


Original shape: (11, 130)
New shape: (11, 65)

First few columns:
['service', 'Fall 2014', 'Fall 2014.2', 'Spring 2015', 'Spring 2015.2', 'Summer 2015', 'Summer 2015.2', 'Fall 2015', 'Fall 2015.2', 'Spring 2016']


After removing the unnecessary columns, the dataset now has 11 rows and 65 columns, reduced from the original 130 columns. This makes the data much easier to work with and interpret. The remaining columns focus only on students who actually received services, combining both Directed and Exempted service recipients for each academic term.

## Convert String Numbers to Integers

In [8]:
#I now have to convert string numbers to integers. I have strings like 5,973 and NaN values.

# STEP 1: Get all term columns
# These are all columns except the "service" column
term_columns = []

for col in df_services_clean.columns:
    if col != "service":
        term_columns.append(col)

print("Number of term columns to convert:", len(term_columns))
print("\n" + "=" * 60)

# STEP 2: Convert each term column from text to numbers
for col in term_columns:

    # Some numbers look like "5,973" and are stored as text
    # First, turn everything into text and remove commas
    df_services_clean[col] = (
        df_services_clean[col]
        .astype(str)
        .str.replace(",", "")
    )

    # Convert text to numbers
    # If conversion fails, pandas will use NaN
    df_services_clean[col] = pd.to_numeric(
        df_services_clean[col],
        errors="coerce"
    )

    # Replace NaN with 0
    # This means no students received the service
    df_services_clean[col] = df_services_clean[col].fillna(0).astype(int)

print("Conversion complete!")
print("\n" + "=" * 60)

# Explain what changed
print("Before: values looked like '5,973' or were blank")
print("After: values are numbers like 5973 or 0")
print("\n" + "=" * 60)

# Show the first few rows to confirm
print("First 3 rows with numeric values:")
print(df_services_clean.head(3))

print("\n" + "=" * 60)

# Show data types to confirm conversion worked
print("Data types of the first few columns:")
print(df_services_clean.dtypes[:10])


Number of term columns to convert: 64

Conversion complete!

Before: values looked like '5,973' or were blank
After: values are numbers like 5973 or 0

First 3 rows with numeric values:
                                             service  Fall 2014  Fall 2014.2  \
0  Credit - Academic/Progress Probation Services ...         16            0   
1  Credit - Counseling/Advisement Services       ...       5973            0   
2  Credit - Education Plan Services              ...       3860            0   

   Spring 2015  Spring 2015.2  Summer 2015  Summer 2015.2  Fall 2015  \
0          152              0           71              0        176   
1         7908              0         3453              0      13047   
2         6025              0         2787              0       9097   

   Fall 2015.2  Spring 2016  Spring 2016.2  Summer 2016  Summer 2016.2  \
0            0          102              0           29              0   
1            0        11637              0         5462 

## Standardize Service Names

In [16]:
#Next is to Standardize Service Names. The inconsistent trailling spaces and dots is problematic. This will make it harder to filter and group.
# Clean service names
# Step 1: Strip whitespace
df_services_clean['service'] = df_services_clean['service'].str.strip()

# Step 2: Remove trailing dots and ellipsis
df_services_clean['service'] = df_services_clean['service'].str.rstrip('.')

# Step 3: Strip again (in case there were spaces after dots)
df_services_clean['service'] = df_services_clean['service'].str.strip()

# After cleaning
print("\nAFTER cleaning:")
print("="*60)
print(df_services_clean['service'].tolist()[:3])
print("\n" + "="*60)
print("\nAll service names:")
for i, service in enumerate(df_services_clean['service'], 1):
    print(f"{i}. {service}")





AFTER cleaning:
['Credit - Academic/Progress Probation Services', 'Credit - Counseling/Advisement Services', 'Credit - Education Plan Services']


All service names:
1. Credit - Academic/Progress Probation Services
2. Credit - Counseling/Advisement Services
3. Credit - Education Plan Services
4. Credit - Initial Assessment Services Placement
5. Credit - Initial Orientation Services
6. Credit - Other Services
7. Non-Credit - Counseling/Advisement Services
8. Non-Credit - Education Plan Services
9. Non-Credit - Initial Assessment Services Placement
10. Non-Credit - Initial Orientation Services
11. Non-Credit - Other Services


## Reshape from Wide to Long Format

In [17]:
#Wide format is hard to analyze so I will convert it to long format which means each row = one service in one term.
#So instead of Counseling | Summer 2014 | Fall 2014 | Spring 2015 | ... I'll get Counseling | Summer 2014 | 8312
#Counseling | Fall 2014   | 53990
#Counseling | Spring 2015 | 49810

# Convert the dataset from wide format to long format
df_services_long = df_services_clean.melt(
    id_vars=["service"],      # Keep 'service' as the main identifier
    var_name="term_year",     
    value_name="count"        # Student counts go into this column
)


In [18]:
# Compare dataset size before and after reshaping
print("Original shape (wide format):", df_services_clean.shape)
print("New shape (long format):", df_services_long.shape)

print("\n" + "=" * 60)

# Preview the transformed data
print("First 10 rows of long-format data:")
df_services_long.head(10)

print("\n" + "=" * 60)

# Sanity check on total rows
print(f"Total rows: {len(df_services_long):,}")
print("This equals: number of services × number of academic terms")


Original shape (wide format): (11, 65)
New shape (long format): (704, 3)

First 10 rows of long-format data:

Total rows: 704
This equals: number of services × number of academic terms


## Parse Term and Yea Information

In this step, I will clean and break apart the term_year column, which originally combines multiple pieces of information into a single string (e.g., "Fall 2014" or "Fall 2014.2"). To make the data easier to analyze, we extract three separate variables:

Term — the academic term (Fall, Spring, or Summer)

Year — the calendar year (e.g., 2014, 2015)

Population — whether the data represents Directed students or Exempted students. The .2 suffix in the column name indicates Exempted students. If the suffix is not present, the data is classified as Directed. After identifying the population type, we remove the suffix and use pattern matching to extract the term and year.This step transforms a single, messy text column into clearly defined fields that can be grouped, filtered, and analyzed independently. Parsing strings like this is a common and important data-cleaning task, especially when working with institutional datasets that encode multiple meanings into column names.

In [21]:
# Population: '.2' means Exempted, otherwise Directed
df_services_long["population"] = np.where(
    df_services_long["term_year"].str.contains(".2", regex=False),
    "Exempted",
    "Directed"
)


# Remove suffix and split term/year
clean_term_year = df_services_long["term_year"].str.replace(".2", "", regex=False)
df_services_long[["term", "year"]] = clean_term_year.str.split(expand=True)
df_services_long["year"] = df_services_long["year"].astype(int)

# Quick sanity check
df_services_long[["term", "year", "population"]].drop_duplicates().head()


,term,year,population
0,Fall,2014,Directed
11,Fall,2014,Exempted
22,Spring,2015,Directed
33,Spring,2015,Exempted
44,Summer,2015,Directed


## Map to Academic Year and Aggregate

Student Success Services are reported by term (Fall, Spring, Summer). But Program Awards, Headcount, and FTES are reported by academic year so i need to aggregate terms to academic years. Mapping rules:

Fall 2014 → Academic Year 2014-2015
Spring 2015 → Academic Year 2014-2015
Summer 2015 → Academic Year 2014-2015
(Fall starts the academic year, Spring and Summer are part of that year)

Then we sum: All three terms for each service in each academic year.

Key Learning: Temporal alignment is critical when merging datasets!

In [24]:
df_services_long["academic_year"] = np.where(
    df_services_long["term"] == "Fall",
    df_services_long["year"].astype(str) + "-" + (df_services_long["year"] + 1).astype(str),
    (df_services_long["year"] - 1).astype(str) + "-" + df_services_long["year"].astype(str)
)


In [25]:
df_services_annual = (
    df_services_long
    .groupby(["academic_year", "service", "population"], as_index=False)["count"]
    .sum()
    .sort_values(["academic_year", "service", "population"])
)


In [26]:
df_services_annual.head()


,academic_year,service,population,count
0,2014-2015,Credit - Academic/Progress Probation Services,Directed,239
1,2014-2015,Credit - Academic/Progress Probation Services,Exempted,0
2,2014-2015,Credit - Counseling/Advisement Services,Directed,17334
3,2014-2015,Credit - Counseling/Advisement Services,Exempted,0
4,2014-2015,Credit - Education Plan Services,Directed,12672


This output confirms that the term-level Student Success Services data was successfully converted into annual totals by mapping Fall terms to the current academic year and Spring/Summer terms to the prior academic year. Counts were then summed across all three terms for each service and population group. The resulting dataset contains 242 observations representing all combinations of academic year (2014–2015 through 2024–2025), service category, and population (Directed vs. Exempted). This annualized structure aligns with the temporal granularity of Program Awards, Headcount, and FTES data, making it suitable for correlation and regression analysis.

## Save Cleaned Annual Services Data

In [28]:
# Export cleaned annual services data
output_file = 'cleaned_services_annual.csv'
df_services_annual.to_csv(output_file, index=False)

print("\nColumn names:")
print(df_services_annual.head())


Column names:
  academic_year                                        service population  \
0     2014-2015  Credit - Academic/Progress Probation Services   Directed   
1     2014-2015  Credit - Academic/Progress Probation Services   Exempted   
2     2014-2015        Credit - Counseling/Advisement Services   Directed   
3     2014-2015        Credit - Counseling/Advisement Services   Exempted   
4     2014-2015               Credit - Education Plan Services   Directed   

   count  
0    239  
1      0  
2  17334  
3      0  
4  12672  


The cleaning pipeline systematically removed metadata rows, extracted only the "Service Received" columns for both Directed and Exempted student populations, converted string-encoded numbers to integers, standardized service names, reshaped the data from wide to long format (704 term-level observations), parsed temporal information to identify term/year/population, mapped all terms to their corresponding academic years (Fall 2014 → 2014-2015, Spring/Summer 2015 → 2014-2015), aggregated the three terms per year into annual totals for each of the 11 service types (counseling, education planning, orientation, assessment, probation, and "other" services for both credit and non-credit programs), and exported the final dataset as cleaned_services_annual.csv with a clear 4-column structure (academic_year, service, population, count) that perfectly aligns with the temporal granularity of the Program Awards, Headcount, and FTES datasets we still need to clean. 

## Extract and Clean Student Headcount Data

Next is read raw file and extract column names from the Student Headcount files from 2014 to 2025. I will create clean dataframe with academic_year and student_count.

In [30]:
#Read raw file
headcount_raw = pd.read_csv('2014_to_2025_StudentHeadcount.csv')

# Extract column names
col_names = headcount_raw.columns.tolist()
print("Number of columns:", len(col_names))
print("Number of rows:", len(headcount_raw))

# Get data row - it's the LAST row (index 1 in this case)
# Row 0 = "Student Count, Student Count (%)" labels
# Row 1 = "San Francisco CCD" with actual data
data_row = headcount_raw.iloc[-1]  # Use -1 to get last row (more reliable)

print("\nData row:")
print(data_row)

# Create lists for years and counts
years = []
counts = []

# Extract every other column starting from index 1 (Student Count columns)
# Pattern: col 1, 3, 5, 7, 9... are Student Counts
# Pattern: col 2, 4, 6, 8, 10... are Student Count (%) - we skip these
for i in range(1, len(col_names), 2):
    year = col_names[i].replace('Annual ', '')
    count_str = str(data_row.iloc[i])
    
    # Remove commas and convert to integer
    if count_str != 'nan' and count_str != '':
        count = int(count_str.replace(',', ''))
        years.append(year)
        counts.append(count)
        print(f"Extracted: {year} = {count}")

# Create clean dataframe
headcount_clean = pd.DataFrame({
    'academic_year': years,
    'student_count': counts
})

print("\n=== Cleaned Headcount Data ===")
print(headcount_clean)
print("\nShape:", headcount_clean.shape)
print("\nData types:")
print(headcount_clean.dtypes)

Number of columns: 23
Number of rows: 2

Data row:
Unnamed: 0            San Francisco CCD
Annual 2014-2015                 70,197
Annual 2014-2015.1              100.00%
Annual 2015-2016                 65,855
Annual 2015-2016.1              100.00%
Annual 2016-2017                 63,345
Annual 2016-2017.1              100.00%
Annual 2017-2018                 69,415
Annual 2017-2018.1              100.00%
Annual 2018-2019                 69,834
Annual 2018-2019.1              100.00%
Annual 2019-2020                 61,610
Annual 2019-2020.1              100.00%
Annual 2020-2021                 38,798
Annual 2020-2021.1              100.00%
Annual 2021-2022                 37,830
Annual 2021-2022.1              100.00%
Annual 2022-2023                 39,532
Annual 2022-2023.1              100.00%
Annual 2023-2024                 44,020
Annual 2023-2024.1              100.00%
Annual 2024-2025                 46,308
Annual 2024-2025.1              100.00%
Name: 1, dtype: object
Extrac

I also need to check for missing values and anomalies. The reason is I know there might be some issues during COVID and I want to make sure I exclude it if needed. 

In [35]:
# Check for missing values
print("Missing values:")
print(headcount_clean.isnull().sum())

# Check value ranges
print("\nStudent count statistics:")
print(headcount_clean['student_count'].describe())

# Check for anomalies - calculate year-over-year changes
print("\nYear-over-year changes:")
headcount_clean['yoy_change'] = headcount_clean['student_count'].diff()
headcount_clean['yoy_pct_change'] = headcount_clean['student_count'].pct_change() * 100

print(headcount_clean[['academic_year', 'student_count', 'yoy_change', 'yoy_pct_change']])

# Drop temporary columns before export
headcount_clean = headcount_clean[['academic_year', 'student_count']]

# Export cleaned data
headcount_clean.to_csv('cleaned_headcount.csv', index=False)

print("\n Cleaned headcount data exported to: cleaned_headcount.csv")
print(f" Shape: {headcount_clean.shape[0]} academic years, {headcount_clean.shape[1]} columns")


Missing values:
academic_year     0
student_count     0
yoy_change        1
yoy_pct_change    1
dtype: int64

Student count statistics:
count       11.000000
mean     55158.545455
std      13719.906081
min      37830.000000
25%      41776.000000
50%      61610.000000
75%      67635.000000
max      70197.000000
Name: student_count, dtype: float64

Year-over-year changes:
   academic_year  student_count  yoy_change  yoy_pct_change
0      2014-2015          70197         NaN             NaN
1      2015-2016          65855     -4342.0       -6.185450
2      2016-2017          63345     -2510.0       -3.811404
3      2017-2018          69415      6070.0        9.582445
4      2018-2019          69834       419.0        0.603616
5      2019-2020          61610     -8224.0      -11.776499
6      2020-2021          38798    -22812.0      -37.026457
7      2021-2022          37830      -968.0       -2.494974
8      2022-2023          39532      1702.0        4.499075
9      2023-2024          4

The cleaned headcount data reveals significant enrollment volatility at City College of San Francisco over the 11-year study period (2014-2025), with no missing values across all academic years. Starting at 70,197 students in 2014-2015, enrollment remained relatively stable through 2019-2020 (averaging ~66,000 students), but then experienced a catastrophic 37% decline during the 2020-2021 academic year due to COVID-19, plummeting to just 38,798 students—the lowest point in the entire dataset. While enrollment has been recovering steadily since then with year-over-year increases of 4.5%, 11.4%, and 5.2%, the 2024-2025 enrollment of 46,308 students remains 34% below the 2014-2015 peak, indicating that CCSF has not yet returned to pre-pandemic enrollment levels. This dramatic enrollment shift will be critical context for analyzing service participation rates and program completion trends, as raw service counts must be normalized by student headcount to accurately assess institutional performance over time.

## FTES and Why I decided to exclude this 

While Full-Time Equivalent Students (FTES) data was collected and clean for this study, I ultimately excluded it from the final analysis because of methodological mismatch with my research question. FTES is a unit-load metric(where 15 units = 1.0 FTES) designed primarily for state funding calculations, measuring the aggregate instructional burden rather than individual student outcomes; in contrast, this study examines whether student support service participation increases the likelihood that individual students will complete programs or certificates, which is inherently a person-based outcome rather than a unit-based measure. Using FTES as a normalization metric would conflate two different units of analysis—instructional load versus individual student success—creating an "apples to oranges" comparison that obscures rather than illuminates the relationship between service utilization and program completion.  Student headcount provides the methodologically appropriate denominator for this research because both the independent variables (service participation counts) and dependent variable (program completions) are person-based metrics: students either do or do not receive services, and students either do or do not complete programs, making headcount-normalized rates (per 1,000 students) the correct approach for examining this research question. 

## Load and Clean Program Awards Data

The Program Awards datasets from 2014–2025 are very large, so they were downloaded as four separate CSV files, each representing a different program discipline category. To analyze overall institutional outcomes, these files must be combined and aggregated to produce total awards across all degrees and certificates.

Input Files

1progawards.csv (Category 1)

2progawards.csv (Category 2)

3progawards.csv (Category 3)

4progawards.csv (Category 4)

Key Structure of Each File

Row 0 contains “San Francisco Total”, which reports total awards for the institution by academic year

Subsequent rows break awards down by credential type (A.A., A.S., certificates, etc.)

Columns represent academic years (Annual 2014–2015 through Annual 2024–2025)

Processing Goal

Extract the San Francisco Total row from each file

Sum totals across all four discipline categories

Produce a single institutional awards time series

Output

cleaned_program_awards.csv

11 rows × 2 columns

academic_year

total_awards

This final dataset represents total institutional program awards per academic year, aligned with the Student Success Services, Headcount, and FTES datasets for longitudinal analysis.

In [39]:
# Load all 4 program awards files
award_file_1 = pd.read_csv('1progawards.csv')
award_file_2 = pd.read_csv('2progawards.csv')
award_file_3 = pd.read_csv('3progawards.csv')
award_file_4 = pd.read_csv('4progawards.csv')

In [41]:
for i, df in enumerate([award_file_1, award_file_2, award_file_3, award_file_4], start=1):
    print(f"File {i} shape:", df.shape)
    display(df.head(3))



File 1 shape: (15, 13)


,Unnamed: 0,Unnamed: 1,Annual 2014-2015,Annual 2015-2016,Annual 2016-2017,Annual 2017-2018,Annual 2018-2019,Annual 2019-2020,Annual 2020-2021,Annual 2021-2022,Annual 2022-2023,Annual 2023-2024,Annual 2024-2025
0,San Francisco Total,NaN,306.0,279.0,250.0,249.0,293.0,334.0,874.0,810.0,737.0,644.0,652.0
1,NaN,Associate in Science for Transfer (A.S.-T) Deg...,13.0,36.0,44.0,58.0,64.0,87.0,93.0,114.0,89.0,105.0,120.0
2,NaN,Associate of Science (A.S.) degree ...,101.0,65.0,66.0,54.0,82.0,85.0,121.0,177.0,152.0,144.0,145.0


File 2 shape: (13, 13)


,Unnamed: 0,Unnamed: 1,Annual 2014-2015,Annual 2015-2016,Annual 2016-2017,Annual 2017-2018,Annual 2018-2019,Annual 2019-2020,Annual 2020-2021,Annual 2021-2022,Annual 2022-2023,Annual 2023-2024,Annual 2024-2025
0,San Francisco Total,NaN,796.0,625.0,585.0,623.0,"1,089","1,576","1,162","1,241","1,605","1,433","1,499"
1,NaN,Associate in Science for Transfer (A.S.-T) Deg...,12.0,7.0,13.0,17.0,23,28,27,34,25,17,17
2,NaN,Associate in Arts for Transfer (A.A.-T) Degree...,3.0,5.0,4.0,9.0,21,26,34,29,33,31,21


File 3 shape: (20, 13)


,Unnamed: 0,Unnamed: 1,Annual 2014-2015,Annual 2015-2016,Annual 2016-2017,Annual 2017-2018,Annual 2018-2019,Annual 2019-2020,Annual 2020-2021,Annual 2021-2022,Annual 2022-2023,Annual 2023-2024,Annual 2024-2025
0,San Francisco Total,NaN,"1,832","1,486","1,360","1,471","1,898","1,803","2,952","4,109","3,562","3,456","3,597"
1,NaN,Associate in Science for Transfer (A.S.-T) Deg...,14,18,14,18,6,3,4,2,1,3,3
2,NaN,Associate in Arts for Transfer (A.A.-T) Degree...,16,20,19,16,19,26,35,37,36,25,28


File 4 shape: (11, 13)


,Unnamed: 0,Unnamed: 1,Annual 2014-2015,Annual 2015-2016,Annual 2016-2017,Annual 2017-2018,Annual 2018-2019,Annual 2019-2020,Annual 2020-2021,Annual 2021-2022,Annual 2022-2023,Annual 2023-2024,Annual 2024-2025
0,San Francisco Total,NaN,888.0,790.0,690.0,768.0,857.0,941.0,629.0,642.0,500.0,691.0,813.0
1,NaN,Associate in Science for Transfer (A.S.-T) Deg...,48.0,66.0,66.0,64.0,90.0,88.0,83.0,70.0,71.0,67.0,71.0
2,NaN,Associate in Arts for Transfer (A.A.-T) Degree...,93.0,112.0,101.0,112.0,165.0,152.0,212.0,205.0,154.0,186.0,184.0


The four Program Awards files all share the same structure, with academic years (2014–2015 through 2024–2025) as columns and award categories as rows. In each file, row 0 (“San Francisco Total”) contains the institutional award totals for that category and year, which is the row used for aggregation.

While the files are structurally consistent, they differ in size and data formatting. The number of award types varies by category, and award counts are stored as a mix of numeric values and strings with commas. This inconsistency is handled later in the cleaning process by converting all values to numeric format.

Across categories, award volumes vary substantially. One category is much larger than the others, contributing the majority of awards each year, while the remaining categories contribute more modest but consistent totals. When all four categories are combined, CCSF awards approximately 3,000 to 7,000 degrees and certificates per academic year.

This confirms that summing the four files’ “San Francisco Total” rows will produce a complete and accurate measure of total institutional awards over time.

In [48]:
# Function to clean a single program awards file
def clean_program_awards_file(filepath, file_num):
    """
    Extract and clean the San Francisco Total row from a program awards file.
    
    Parameters:
    - filepath: path to CSV file
    - file_num: file number (for tracking)
    
    Returns:
    - DataFrame with academic_year and awards columns
    """
    # Load file
    df = pd.read_csv(filepath)
    
    # Get the San Francisco Total row (row 0)
    total_row = df.iloc[0]
    
    # Get column names
    col_names = df.columns.tolist()
    
    # Extract years and award counts
    years = []
    awards = []
    
    # Skip first 2 columns (empty and label columns)
    # Start from column 2 (Annual 2014-2015)
    for i in range(2, len(col_names)):
        year = col_names[i].replace('Annual ', '')
        award_str = str(total_row.iloc[i])
        
        # Convert to integer (handle commas, floats, and missing values)
        if award_str != 'nan' and award_str != '':
            # Remove commas, convert to float first, then to int
            # This handles both "1,089" and "306.0" formats
            award = int(float(award_str.replace(',', '')))
            years.append(year)
            awards.append(award)
            print(f"File {file_num} - {year}: {award} awards")
    
    # Create dataframe
    df_clean = pd.DataFrame({
        'academic_year': years,
        f'awards_file_{file_num}': awards
    })
    
    return df_clean

# Clean each file
print("=== Cleaning File 1 ===")
awards_1 = clean_program_awards_file('1progawards.csv', 1)
print(awards_1)

print("\n=== Cleaning File 2 ===")
awards_2 = clean_program_awards_file('2progawards.csv', 2)
print(awards_2)

print("\n=== Cleaning File 3 ===")
awards_3 = clean_program_awards_file('3progawards.csv', 3)
print(awards_3)

print("\n=== Cleaning File 4 ===")
awards_4 = clean_program_awards_file('4progawards.csv', 4)
print(awards_4)

=== Cleaning File 1 ===
File 1 - 2014-2015: 306 awards
File 1 - 2015-2016: 279 awards
File 1 - 2016-2017: 250 awards
File 1 - 2017-2018: 249 awards
File 1 - 2018-2019: 293 awards
File 1 - 2019-2020: 334 awards
File 1 - 2020-2021: 874 awards
File 1 - 2021-2022: 810 awards
File 1 - 2022-2023: 737 awards
File 1 - 2023-2024: 644 awards
File 1 - 2024-2025: 652 awards
   academic_year  awards_file_1
0      2014-2015            306
1      2015-2016            279
2      2016-2017            250
3      2017-2018            249
4      2018-2019            293
5      2019-2020            334
6      2020-2021            874
7      2021-2022            810
8      2022-2023            737
9      2023-2024            644
10     2024-2025            652

=== Cleaning File 2 ===
File 2 - 2014-2015: 796 awards
File 2 - 2015-2016: 625 awards
File 2 - 2016-2017: 585 awards
File 2 - 2017-2018: 623 awards
File 2 - 2018-2019: 1089 awards
File 2 - 2019-2020: 1576 awards
File 2 - 2020-2021: 1162 awards
File 2

Program awards data were cleaned by extracting the San Francisco Total row from each of four DataMart CSV files covering academic years 2014–2015 through 2024–2025. For each file, the function iterates through the annual award columns, converts award counts from string format to integers (handling commas, floats, and missing values), and constructs a clean dataset containing one row per academic year. This process produced four standardized time-series tables, each representing a distinct award category, and confirmed consistent year-to-year trends with notable increases during the COVID-19 period (2020–2022). The resulting datasets provide a reliable foundation for longitudinal analysis and correlation with student support service participation.

## Calculate Total Award Across all Categories

Summing across all four category columns to get the total institutional awards for each year.

In [51]:
# Calculate total awards (sum across all 4 files)
awards_merged = awards_1.merge(awards_2, on='academic_year', how='outer')
awards_merged = awards_merged.merge(awards_3, on='academic_year', how='outer')
awards_merged = awards_merged.merge(awards_4, on='academic_year', how='outer')

print("=== Merged Awards by Category ===")
print(awards_merged)
print("\nShape:", awards_merged.shape)

# Check for any missing values
print("\nMissing values:")
print(awards_merged.isnull().sum())


=== Merged Awards by Category ===
   academic_year  awards_file_1  awards_file_2  awards_file_3  awards_file_4
0      2014-2015            306            796           1832            888
1      2015-2016            279            625           1486            790
2      2016-2017            250            585           1360            690
3      2017-2018            249            623           1471            768
4      2018-2019            293           1089           1898            857
5      2019-2020            334           1576           1803            941
6      2020-2021            874           1162           2952            629
7      2021-2022            810           1241           4109            642
8      2022-2023            737           1605           3562            500
9      2023-2024            644           1433           3456            691
10     2024-2025            652           1499           3597            813

Shape: (11, 5)

Missing values:
academic_

## Validate and Export Cleaned Program Awards

Next is to verify data quality, calculate year-over-year changes, and then export to CSV

In [54]:
# Check for missing values
print("Missing values:")
print(awards_clean.isnull().sum())

# Check value ranges
print("\nTotal awards statistics:")
print(awards_clean['total_awards'].describe())

# Calculate year-over-year changes
print("\nYear-over-year changes:")
awards_clean['yoy_change'] = awards_clean['total_awards'].diff()
awards_clean['yoy_pct_change'] = awards_clean['total_awards'].pct_change() * 100

print(awards_clean[['academic_year', 'total_awards', 'yoy_change', 'yoy_pct_change']])

# Compare to headcount to calculate completion rate
try:
    headcount = pd.read_csv('cleaned_headcount.csv')
    comparison = awards_clean[['academic_year', 'total_awards']].merge(
        headcount[['academic_year', 'student_count']], 
        on='academic_year'
    )
    comparison['completion_rate_per_1000'] = (comparison['total_awards'] / comparison['student_count']) * 1000
    
    print("\n=== Completion Rate Analysis ===")
    print(comparison[['academic_year', 'student_count', 'total_awards', 'completion_rate_per_1000']])
    print(f"\nAverage completion rate: {comparison['completion_rate_per_1000'].mean():.1f} per 1000 students")
    
except FileNotFoundError:
    print("\nHeadcount file not found - skipping completion rate calculation")

# Drop temporary columns before export
awards_clean = awards_clean[['academic_year', 'total_awards']]

# Export cleaned data
awards_clean.to_csv('cleaned_program_awards.csv', index=False)

print("\n Cleaned program awards data exported to: cleaned_program_awards.csv")
print(f" Shape: {awards_clean.shape[0]} academic years, {awards_clean.shape[1]} columns")



Missing values:
academic_year    0
total_awards     0
dtype: int64

Total awards statistics:
count      11.000000
mean     4854.272727
std      1513.490013
min      2885.000000
25%      3501.000000
50%      4654.000000
75%      6314.000000
max      6802.000000
Name: total_awards, dtype: float64

Year-over-year changes:
   academic_year  total_awards  yoy_change  yoy_pct_change
0      2014-2015          3822         NaN             NaN
1      2015-2016          3180      -642.0      -16.797488
2      2016-2017          2885      -295.0       -9.276730
3      2017-2018          3111       226.0        7.833622
4      2018-2019          4137      1026.0       32.979749
5      2019-2020          4654       517.0       12.496978
6      2020-2021          5617       963.0       20.691878
7      2021-2022          6802      1185.0       21.096671
8      2022-2023          6404      -398.0       -5.851220
9      2023-2024          6224      -180.0       -2.810743
10     2024-2025          6561

Data Quality and Completion Rate Interpretation

The program awards data demonstrate strong overall data quality. All 11 academic years from 2014–2015 through 2024–2025 are present, and there are no missing values in either the academic year or total awards fields, indicating that the data extraction and cleaning process was successful. Descriptive statistics show an average of approximately 4,854 awards per year, with expected year-to-year variation and a clear upward trend following the COVID-19 period.

What initially appears anomalous is the high average completion rate of 99.8 awards per 1,000 students, which increases dramatically during and after the pandemic years. After verifying the figures directly against the California Community Colleges Chancellor’s Office DataMart Program Awards reports, the counts were confirmed to be accurate and correctly downloaded.

Several factors likely explain this elevated completion rate. First, the DataMart “Program Awards” category encompasses more than traditional AA/AS degrees. It includes a broad range of awards such as noncredit certificates, short-term and skills certificates, and Chancellor’s Office–approved certificates. Examination of the underlying files shows a substantial volume of noncredit and short-term awards, which can significantly inflate total completions relative to headcount.

Second, COVID-19 had a pronounced denominator effect. In 2020–2021, enrollment declined sharply to approximately 38,800 students, while awards increased to 5,617. This resulted in a spike in the completion rate from 75.5 to 144.8 awards per 1,000 students. Pandemic-related policy changes, accelerated credential pathways, emergency workforce certificates, and increased flexibility in completion requirements likely contributed to this surge.

Finally, while City College of San Francisco may outperform peer institutions in certain completion outcomes, rates of this magnitude exceed typical statewide and national community college benchmarks. This suggests the pattern is driven primarily by award definitions and enrollment dynamics rather than uniquely exceptional institutional performance alone.

Taken together, these findings indicate that the high completion rate is valid but must be interpreted cautiously. Subsequent analyses will distinguish between award types and consider alternative normalization strategies to ensure meaningful comparisons over time.

## Merging all datasets

Here is where I merge all the three cleaned datasets (services, headcount, program awards) into a single master dataset, aligned by academic year.But first, I need to make sure I transform the services data from long format to wide format. I will also focus on Directed students only since they are the target population for my research.

In [62]:
import pandas as pd

# 1) Load the long services file
services = pd.read_csv("cleaned_services_annual.csv")

# 2) Keep Directed only
services = services[services["population"] == "Directed"].copy()

# 3) Map long service names to short column names
service_map = {
    "Credit - Counseling/Advisement Services": "counseling",
    "Credit - Education Plan Services": "ed_plan",
    "Credit - Initial Orientation Services": "orientation",
    "Credit - Initial Assessment Services Placement": "assessment",
    "Credit - Academic/Progress Probation Services": "probation",
    "Credit - Other Services": "other_services",
    "Non-Credit - Counseling/Advisement Services": "noncredit_counseling",
    "Non-Credit - Education Plan Services": "noncredit_ed_plan",
    "Non-Credit - Initial Orientation Services": "noncredit_orientation",
    "Non-Credit - Initial Assessment Services Placement": "noncredit_assessment",
    "Non-Credit - Other Services": "noncredit_other",
}

services["service_short"] = services["service"].map(service_map)

# Optional safety check (shows any service names you forgot to map)
unmapped = services[services["service_short"].isna()]["service"].unique()
print("Unmapped services:", unmapped)

# 4) Pivot to wide (one row per year)
services_wide = (
    services.pivot_table(
        index="academic_year",
        columns="service_short",
        values="count",
        aggfunc="sum",
        fill_value=0
    )
    .reset_index()
)

# Make service columns integers
service_cols = [c for c in services_wide.columns if c != "academic_year"]
services_wide[service_cols] = services_wide[service_cols].astype(int)

# 5) Add total services
services_wide["total_services"] = services_wide[service_cols].sum(axis=1)

print("Shape:", services_wide.shape)
services_wide.head()



Unmapped services: []
Shape: (11, 13)


service_short,academic_year,assessment,counseling,ed_plan,noncredit_assessment,noncredit_counseling,noncredit_ed_plan,noncredit_orientation,noncredit_other,orientation,other_services,probation,total_services
0,2014-2015,7952,17334,12672,0,0,0,0,0,5866,15505,239,59568
1,2015-2016,12146,30146,22125,0,0,0,0,0,8273,20572,307,93569
2,2016-2017,10723,20325,11992,33,33,0,0,394,8415,13438,100,65453
3,2017-2018,14062,34075,22080,0,2,0,0,475,10049,26504,234,107481
4,2018-2019,15408,25347,17923,0,1,0,0,1261,6826,29072,305,96143


In [66]:
import pandas as pd

# Load year-level datasets
headcount = pd.read_csv("cleaned_headcount.csv")
awards = pd.read_csv("cleaned_program_awards.csv")

# Merge everything into one modeling table
master_df = services_wide.merge(headcount, on="academic_year", how="left")
master_df = master_df.merge(awards, on="academic_year", how="left")

# Quick checks
print("Shape:", master_df.shape)
print("\nMissing values per column:")
print(master_df.isna().sum())

master_df.head()



Shape: (11, 15)

Missing values per column:
academic_year            0
assessment               0
counseling               0
ed_plan                  0
noncredit_assessment     0
noncredit_counseling     0
noncredit_ed_plan        0
noncredit_orientation    0
noncredit_other          0
orientation              0
other_services           0
probation                0
total_services           0
student_count            0
total_awards             0
dtype: int64


,academic_year,assessment,counseling,ed_plan,noncredit_assessment,noncredit_counseling,noncredit_ed_plan,noncredit_orientation,noncredit_other,orientation,other_services,probation,total_services,student_count,total_awards
0,2014-2015,7952,17334,12672,0,0,0,0,0,5866,15505,239,59568,70197,3822
1,2015-2016,12146,30146,22125,0,0,0,0,0,8273,20572,307,93569,65855,3180
2,2016-2017,10723,20325,11992,33,33,0,0,394,8415,13438,100,65453,63345,2885
3,2017-2018,14062,34075,22080,0,2,0,0,475,10049,26504,234,107481,69415,3111
4,2018-2019,15408,25347,17923,0,1,0,0,1261,6826,29072,305,96143,69834,4137


In [68]:
import numpy as np
import pandas as pd


not_service_cols = ["academic_year", "student_count", "total_awards"]
service_cols = [c for c in master_df.columns if c not in not_service_cols]

# Create service rates per 1,000 students
for c in service_cols:
    master_df[c + "_rate"] = (master_df[c] / master_df["student_count"]) * 1000

# Create award rate per 1,000 students
master_df["award_rate"] = (master_df["total_awards"] / master_df["student_count"]) * 1000

#  Quality checks 
rate_cols = [c for c in master_df.columns if c.endswith("_rate")] + ["award_rate"]

missing = master_df[rate_cols].isna().sum().sum()
infinite = np.isinf(master_df[rate_cols].to_numpy()).sum()
negative = (master_df[rate_cols] < 0).sum().sum()

# Display sample of final data
print(f"\nSample rows (first 3 years):")
sample_cols = ['academic_year', 'student_count', 'counseling', 'counseling_rate', 
               'ed_plan', 'ed_plan_rate', 'total_awards', 'award_rate']
print(master_df[sample_cols].head(3).to_string(index=False))

print("Rows:", master_df.shape[0], "Columns:", master_df.shape[1])
print("Missing rate values:", int(missing))
print("Infinite rate values:", int(infinite))
print("Negative rate values:", int(negative))

# Export the final dataset for EDA + modeling
output_file = "master_dataset_annual.csv"
master_df.to_csv(output_file, index=False)
print("Saved:", output_file)

# Quick preview of the most important columns
preview_cols = [
    "academic_year", "student_count", "total_awards",
    "total_services", "award_rate"
]
print(master_df[preview_cols].head())



Sample rows (first 3 years):
academic_year  student_count  counseling  counseling_rate  ed_plan  ed_plan_rate  total_awards  award_rate
    2014-2015          70197       17334       246.933630    12672    180.520535          3822   54.446771
    2015-2016          65855       30146       457.763268    22125    335.965378          3180   48.287905
    2016-2017          63345       20325       320.861946    11992    189.312495          2885   45.544242
Rows: 11 Columns: 41
Missing rate values: 0
Infinite rate values: 0
Negative rate values: 0
Saved: master_dataset_annual.csv
  academic_year  student_count  total_awards  total_services  award_rate
0     2014-2015          70197          3822           59568   54.446771
1     2015-2016          65855          3180           93569   48.287905
2     2016-2017          63345          2885           65453   45.544242
3     2017-2018          69415          3111          107481   44.817403
4     2018-2019          69834          4137        